# **Practice 9**

## Boundary value problems

1. We are going to consider the following ODE system:

$$ \begin{array}{l}
y''(t) = -g \\
x''(t)  =  0
\end{array} $$

where $g$ is the gravity. Maybe some of you have recognized this ODE as the one modelling the trajectory of an object that we throw (for example, with a cannon) not considering the friction with air.

If the cannon is placed at $(x,y)=(0,10)$ and we want to hit an object that is placed at $(x,y)=(100,0)$, calculate the initial velocity $y'(0)$ if $x'(0)=3$.

* You have to think about a procedure to solve this problem for a general position of the object.
* You will have to solve the ODE several times. Use any multistep method (with more than 2 steps).

<br>

<br>

2. Solve the following stiff initial vlaue problem

$$y' = −20y + 20 cos(t) − sin(t), \quad 0 \leq t \leq 5, \; y(0) = 0$$

with $h = 0.25$. Then reduce the stepsize till you get a good numerical solution. Compare with the analytic solution $y(t) = -e^{-20t} + cos t$.


From R. Burden book:

*All the methods for approximating the solution to initial-value problems have error terms that involve a higher derivative of the solution of the equation. If the derivative can be reasonably bounded, then the method will have a predictable error bound that can be used to estimate the accuracy of the approximation. Even if the derivative grows as the steps increase, the error can be kept in relative control, provided that the solution also grows in magnitude. Problems frequently arise, however, when the magnitude of the derivative increases but the solution does not. In this situation, the error can grow so large that it dominates the calculations. Initial-value problems for which this is likely to occur are called stiff equations and are quite common, particularly in the study of vibrations, chemical reactions, and electrical circuits.*

*Stiff differential equations are characterized as those whose exact solution has a term of the form $e^{−ct}$ , where $c$ is a large positive constant. This is usually only a part of the solution, called the transient solution. The more important portion of the solution is called the steady-state solution. The transient portion of a stiff equation will rapidly decay to zero
as $t$ increases, but since the n-th derivative of this term has magnitude $c^n e^{−ct}$, the derivative does not decay as quickly. In fact, since the derivative in the error term is evaluated not at t, but at a number between zero and $t$, the derivative terms can increase as $t$ increases (and very rapidly indeed). Fortunately, stiff equations generally can be predicted from the physical problem from which the equation is derived and, with care, the error can be kept under control. The manner in which this is done is considered in this section.*


In [1]:
import numpy as np


def euler_explicit(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float) -> np.ndarray:
    r"""Computes the explicit (forward) Euler method to solve ODEs.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, t-h, t].

    Examples:

        Lets solve the problem

        :math: `$$\begin{array}{l}
                y'=\lambda y \\
                y(0) = 1
                \end{array}$$`

        for :math:`$\lambda = -1$` over the interval :math: `$[0,1]$` for a stepsize `$h=0.1$`.
    """
    t_ = np.arange(t0, t0+t, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u[0] = y0

    for i in range(N-1):
        u[i+1] = u[i] + h * f(u[i], t_[i])

    return u


def euler_explicit_midpoint(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float) -> np.ndarray:
    r"""Computes the explicit (forward) midpoint Euler method to solve ODEs.

    The **explicit midpoint method** is :math: `u_{n+1}=u_{n-1}+2hf\left(t_n,u_n\right)`

    As two initial values are required, if y0_previous is not provided, it is computed with :math: `$y(-h)=y(0)-hf(0,y(0))$`.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, t-h, t].
    """
    t_ = np.arange(t0, t0+t, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u_previous = y0 - h * f(y0, t_[0])
    u[0] = y0

    for i in range(N-1):
        if i == 0:
            u[i+1] = u_previous + 2 * h * f(u[i], t_[i])
        else:
            u[i+1] = u[i-1] + 2 * h * f(u[i], t_[i])

    return u

def secant(f: 'Callable[float]', a: float, b: float, err: float, Nmax: int = 100_000, x0: float = None) -> float:
    r"""Computes Secant method to find roots :math:`f(x)=0`.

    If there are no roots in the interval :math:`[a, b]`, the method will throw an exception.
    This is checked using bolzano's theorem (If :math:`f(a)*f(b) >= 0`).

    To computes the first iteration, it computes the previous value as :math: `a-1`

    Args:
        f (Callable[float]): Function of which we want to find roots :math:`f(x)=0`.
        a (float): Lower bound of the interval.
        b (float): Upper bound of the interval.
        err (float): Tolerance of the result. It assures that the root is in :math:`[x-err, x+err]`. #TODO: Is this the interval?
        Nmax (int): Maximum number of iterations. Defaults to 100_000.
        x0 (float): Initial guess for the root. Defaults :math:`(a+b)/2`.

    Raises:
        ValueError: If, according to Bolzano's theorem, there cannot be roots in :math:`[a, b]`.
        ValueError: If the method, being at least one root in :math:`[a, b]`, fails to to compute the root.

    Returns:
        float: Root x such as f(x)=0 with a tolerance err.
    """
    if f(a)*f(b) >= 0:
        raise ValueError(f'f(a)*f(b) = {f(a)*f(b)} <0. \t No roots in this interval.')

    x_n = x0 if x0 is not None else (a+b)/2  # TODO: Check if there is a better initial guess
    x_previous = a - 1  # TODO: Check if there is a better initial guess
    f_x_n = f(x_n)

    for _ in range(1, Nmax+1):

        f_x_previous = f(x_previous)

        q_n = (f_x_n - f_x_previous) / (x_n - x_previous)
        x_previous = x_n

        x_n = x_n - f_x_n / q_n

        f_x_n = f(x_n)
        if abs(f_x_n) <= err:
            return x_n

    raise ValueError(f'Could not find a root in the interval [{a}, {b}] with tolerance {err} in {Nmax} iterations.')

In [ ]:

y_0 = 10
x_0 = 0

x_t = 100
y_t = 0

# y_dev_0 ? 
x_dev0 = 3

g = 9.8

Then x(t) = 3t
so x(x_t) = 3t = 100 --> t = 100/3

In [ ]:
t0 = 0
t = 100/3

if y_dev = z, z_dev = -g, --> y_f = Euler(z0) = y_t = 0

Then we need secant(euler(z0))

In [ ]:

def z_dev(z):
    return - g

def euler_z0(z0):
    return euler_explicit(z_dev, z0, t0, t, 1e-4)



def secant_euler(euler_z0):
    return secant()

# Ver la hoja 